# Cloud Storage Exfiltration Spike

**Unusual upload volume to cloud storage domains (Google Drive/Dropbox/Box/OneDrive/MEGA).**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **bytes out**, **user/device posture**, **file types**, **DLP/CASB alerts**.


## Investigation Queries

Summarize outbound volume by user and domain; identify top paths/file types if available.


In [ ]:
// KQL
let window=15m;
CommonSecurityLog
| where TimeGenerated > ago(30d)
| where DestinationHostName has_any ('drive.google.com','dropbox.com','box.com','onedrive.live.com','mega.nz')
| summarize UploadBytes=sum(SentBytes) by SourceUserName, SourceIP, DestinationHostName, bin(TimeGenerated, window)
| order by UploadBytes desc


In [ ]:
# Splunk SPL
index=proxy url_domain IN ("drive.google.com","dropbox.com","box.com","onedrive.live.com","mega.nz")
| bucket _time span=15m
| stats sum(bytes_out) as upload_bytes by user src_ip url_domain _time
| sort - upload_bytes


In [ ]:
# Elastic (KQL/EQL)
event.category:network AND url.domain:(drive.google.com OR dropbox.com OR box.com OR onedrive.live.com OR mega.nz)


## Containment & Response

If exfil suspected: block destination, isolate endpoint, revoke sessions, and perform data loss assessment.


In [ ]:
# PowerShell
## Revoke user tokens (AzureAD)
Revoke-AzureADUserAllRefreshToken -ObjectId <USER_OBJECT_ID>


In [ ]:
# PowerShell
## Windows: block specific domains via enterprise proxy/DNS policy (org-specific)
# Document your control-plane action here.
